<a href="https://colab.research.google.com/github/srajabia/Data201_RajabianSam/blob/main/Assignment_2_Data_Wrangling_with_pandas_Data201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 2 – Data Wrangling with pandas
Sam Rajabian
<hr>

### Import libraries and dataset

In [36]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Reben80/Data201/refs/heads/main/Dataset/housing.csv")
df.head()

,listing_id,price,size,bedrooms,neighborhood,type
0,100001,1500000,1280.741760,1.0,Suburb,Townhouse
1,100002,1500000,1406.283113,2.0,Uptown,SingleFamily
2,100003,1500000,4146.825713,6.0,Suburb,MultiFamily
3,100004,1500000,3946.599818,6.0,Suburb,SingleFamily
4,100005,1500000,1243.751760,1.0,Downtown,MultiFamily


### Part A – Core Wrangling (Method Chaining Required)

In [50]:
(df
 .query("price > 250000 and size > 1000")
 .assign(price_per_sqft= df["price"] / df["size"])  #lambda returned incorrect calculations
 .groupby("neighborhood").agg(mean_ppsqft= ("price_per_sqft", "mean"),
                              median_ppsqft= ("price_per_sqft", "median"),
                              n = ("price", "count"))
 .sort_values("mean_ppsqft", ascending=False)
 )


,mean_ppsqft,median_ppsqft,n
neighborhood,,,
Downtown,977.820905,1001.557049,99
Midtown,921.141446,901.992377,92
Suburb,861.917078,836.878644,157
Uptown,860.935608,843.653468,99
Waterfront,849.508891,792.663291,48


### Part B – Translation to dplyr

```{r}
df |>
  filter(price > 250000 & size > 1000) |>
  mutate(price_per_sqft = price / size) |>
  group_by(neighborhood) |>
  summarise(mean_ppsqft = mean(price_per_sqft),
            median_ppsqft = median(price_per_sqft),
            n = n()) |>
  arrange(desc(mean_ppsqft))
```

**Reflection:** <br>
I feel dplyr pipelines are clearer to me than pandas chaining. I think dplyr is easier to read and more consisent in its syntax than pandas. <br>
They both use the same functions and follow the same general stucture, but pandas chaining is not separated by pipes, and the functions feel less intuitive. For example, in .assign you must type price and size a specific way with "df", brackets, and quotes in pandas, whilst dplyr's mutate only requires the column names.     


### Part C – Boolean Logic Debugging

In [58]:
df[(df["price"] > 250000) & (df["size"] > 1000)]

,listing_id,price,size,bedrooms,neighborhood,type
0,100001,1500000,1280.741760,1.0,Suburb,Townhouse
1,100002,1500000,1406.283113,2.0,Uptown,SingleFamily
2,100003,1500000,4146.825713,6.0,Suburb,MultiFamily
3,100004,1500000,3946.599818,6.0,Suburb,SingleFamily
4,100005,1500000,1243.751760,1.0,Downtown,MultiFamily
...,...,...,...,...,...,...
595,100596,1500000,1443.241197,3.0,Midtown,Condo
596,100597,1500000,1083.909714,2.0,Suburb,Condo
597,100598,1500000,1600.126432,1.0,Suburb,SingleFamily
598,100599,1500000,1248.216637,1.0,Waterfront,Condo


`df[df["price"] > 250000 & df["size"] > 1000]` is incorrect because of an operator precedence error, as there are no parenthesis separating `(df["price"] > 250000)` and `df["size"] > 1000`. As such, the code is being interpreted as `250000 & df["size"]` which outputs an error.


**With .query**

In [59]:
df.query("price > 250000 and size > 1000")

,listing_id,price,size,bedrooms,neighborhood,type
0,100001,1500000,1280.741760,1.0,Suburb,Townhouse
1,100002,1500000,1406.283113,2.0,Uptown,SingleFamily
2,100003,1500000,4146.825713,6.0,Suburb,MultiFamily
3,100004,1500000,3946.599818,6.0,Suburb,SingleFamily
4,100005,1500000,1243.751760,1.0,Downtown,MultiFamily
...,...,...,...,...,...,...
595,100596,1500000,1443.241197,3.0,Midtown,Condo
596,100597,1500000,1083.909714,2.0,Suburb,Condo
597,100598,1500000,1600.126432,1.0,Suburb,SingleFamily
598,100599,1500000,1248.216637,1.0,Waterfront,Condo


### Part D – Short Concept Questions

**1. Why must we wrap each condition in parentheses when using & in pandas?**

We must wrap each condition in parenthesis to avoid operator precedence errors.

**2. What is the advantage of method chaining over creating many temporary DataFrames?**

Method chaining improves code readability by making it more concise and organized.

**3. In `.agg(mean_price=("price", "mean"))`, what does "price" represent? What does "mean" represent?**

Price represents the "price" column in the dataframe. Mean represents the mathematical function that should be used on price.

**4. When you `groupby("neighborhood")`, why does neighborhood appear on the left (index) in the result table?**

Grouping by neighborhood separates rows in the dataframe by neighborhood, which are labeled like a title on the left in the result table.

### Extra practice

In [66]:
(df
 .groupby("type").agg(mean_price= ("price", "mean"),
                              median_price= ("price", "median"),
                              count = ("price", "count"))
 )

,mean_price,median_price,count
type,,,
Condo,1500000.0,1500000.0,183
MultiFamily,1500000.0,1500000.0,63
SingleFamily,1500000.0,1500000.0,235
Townhouse,1500000.0,1500000.0,119
